### My first Machine learning project building and deployment using kubernetes 

In [2]:
import pickle 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [3]:
class IrisModelTrainer():
    
    def __init__(self):
        self.model = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        
    def load_data(self):
        iris = load_iris()
        X = iris.data
        y = iris.target
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
    def train_model(self):
        self.model = LogisticRegression(random_state=42, solver='liblinear')
        self.model.fit(self.X_train, self.y_train)
        
    def evaluate_model(self):
        y_pred = self.model.predict(self.X_test)
        f1 = f1_score(self.y_test, y_pred, average='macro')
        return f1
    
    def save(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump(self.model, file) 

In [4]:
if __name__ == "__main__":
    trainer = IrisModelTrainer()
    trainer.load_data()
    trainer.train_model()
    score = trainer.evaluate_model()
    print("F1 Score: ", score)
    trainer.save('iris_model.pkl')

F1 Score:  0.974320987654321


### Application module

In [10]:
import os 
import numpy as np
from typing import List
from fastapi import FastAPI
from pydantic import BaseModel

In [11]:
os.environ["MODEL_FILE_NAME"] = 'iris_model.pkl'

model_file_name = os.getenv("MODEL_FILE_NAME")

In [13]:
# load the trained model 
with open('iris_model.pkl', 'rb') as f:
    model = pickle.load(f)
    
# fast api Application 
app = FastAPI()

In [14]:
# Define input data model
class IrisData(BaseModel):
    septal_length: float
    septal_width: float
    petal_length: float
    petal_width: float
    
# Define output data model
class IrisResponse(BaseModel):
    prediction: str
    
# Define the prediction function 
def predict_iris_species(input_data):
    # convert the input data into numpy array 
    input_array = np.array([input_data.septal_length, input_data.septal_width, input_data.petal_length, input_data.petal_width]).reshape(1, -1)
    
    # make prediction using the loaded model 
    prediction = model.predict(input_array)[0]
    
    # map integer prediction to corresponding species name
    species_map = {
        0 : 'Iris Setosa',
        1 : 'Iris Versicolour',
        2 : 'Iris Verginica'
    }
    
    prediction_name = species_map[prediction]
    
    # return the prediction result 
    return IrisResponse(prediction=prediction_name)   

In [15]:
# Define the end point for single prediction
@app.post("/predict")
def predict_single(data: IrisData):
    return predict_iris_species(data)

In [16]:
# Define the endpoint for batch prediction
@app.post("/predict_batch")
def predict_batch(data: List[IrisData]):
    predictions = [predict_iris_species(item) for item in data]
    return predictions

In [22]:
from pathlib import Path

Path().resolve().as_posix() + '/play'

'/Users/mohamedabokahf/Desktop/Text_web_mining/play'